# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr

In [2]:
scd = True

# Define a submodel

In [3]:
class SubModel(bu.Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

# Test a list

In [32]:
class CrossSectionLayout(bu.ModelDict):
    name = 'Cross Section Layout'
    item_type = SubModel
    
    sum_length = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_sum_length(self):
        print('updating sum')
        return sum( sm.length for sm in self.items.values() ) 

    ipw_view = bu.View(
        bu.Item('sum_length', readonly=True)
    )

In [36]:
class Beam(bu.Model):
    layout = bu.Instance(CrossSectionLayout, ())
    tree = ['layout']

In [39]:
b = Beam()
b.layout = {'one': SubModel()}
b.interact()

TraitError: The 'layout' trait of a Beam instance must be a CrossSectionLayout or None, but a value of {'one': <__main__.SubModel object at 0x7fa12cf0c4f0>} <class 'dict'> was specified.

In [18]:
csl = CrossSectionLayout(state_change_debug=scd)
csl.sum_length

updating sum


0

In [19]:
one1 = SubModel(state_change_debug=scd)
csl['one'] = one1
csl['two'] = SubModel(state_change_debug=scd)

graph_changed <__main__.CrossSectionLayout object at 0x7fa1365c7d10> Notification from child item
graph_changed <__main__.CrossSectionLayout object at 0x7fa1365c7d10> Notification from child item


In [20]:
csl['one'].name

'one'

In [21]:
csl['one'].length = 4

value_changed <__main__.SubModel object at 0x7fa12dbbd4f0> TraitChangeEvent(object=<__main__.SubModel object at 0x7fa12dbbd4f0>, name='length', old=2, new=4.0)
value_changed <__main__.CrossSectionLayout object at 0x7fa1365c7d10> Notification from child <__main__.SubModel object at 0x7fa12dbbd4f0>


In [22]:
csl.sum_length

updating sum


6.0

In [23]:
csl.tree

['one', 'two']

In [24]:
csl.interact()

rebuild tree
ModelDict: get_tree_submodels {'one': <__main__.SubModel object at 0x7fa12dbbd4f0>, 'two': <__main__.SubModel object at 0x7fa12dbd6ef0>}


In [25]:
csl['one'] = SubModel()

graph_changed <__main__.CrossSectionLayout object at 0x7fa1365c7d10> Notification from child item
ModelDict: get_tree_submodels {'one': <__main__.SubModel object at 0x7fa12dbe8130>, 'two': <__main__.SubModel object at 0x7fa12dbd6ef0>}
updating sum


In [26]:
del csl['one']

graph_changed <__main__.CrossSectionLayout object at 0x7fa1365c7d10> Notification from child item
ModelDict: get_tree_submodels {'two': <__main__.SubModel object at 0x7fa12dbd6ef0>}
updating sum


In [27]:
csl['two'].parents

{<__main__.CrossSectionLayout at 0x7fa1365c7d10>}

In [30]:
del one1